In [ ]:
!pip install matplotlib==3.7.5

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from fosforml import *
from fosforml.constants import MLModelFlavours

In [2]:
from refractio import snowflake
cars = snowflake.get_dataframe("CAR_NEW200MB")
cars

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


,BUYING_NUM,MAINTENANCE_NUM,DOORS_NUM,PERSONS_NUM,LUGBOOT_NUM,SAFETY_NUM,CLASS_NUM
0,0,2,2,1,2,1,0
1,1,1,1,1,2,2,0
2,3,1,3,0,1,2,1
3,0,3,1,2,2,0,3
4,3,0,3,1,0,1,0
...,...,...,...,...,...,...,...
119699995,3,1,1,0,0,1,0
119699996,0,0,0,0,0,2,1
119699997,2,2,0,2,2,0,3
119699998,3,3,2,0,2,0,3


In [ ]:
# cars = pd.read_csv("/data/car_new.csv")
# cars

In [ ]:
Y = cars['CLASS_NUM']
X = cars.drop (['CLASS_NUM'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

In [ ]:
import sys
m_size = sys.getsizeof(model)
m_size

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(model, "/notebooks/notebooks/CAR_NEW200MB")

In [ ]:
X_train.head(1)

In [ ]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [ ]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [ ]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

In [ ]:
register_model(model, 
               score, 
               name='car_evaluation_new2', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=False, 
               kyd_score = False
              )

In [ ]:
!pip install snowflake[ml]==0.8.0

In [1]:
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry as model_registry
import time

ModuleNotFoundError: No module named 'snowflake.snowpark'